# Imports

In [1]:
import os

import numpy as np
import tensorflow as tf

import config, music_model

2022-12-06 19:50:02.733714: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-06 19:50:03.391170: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/volpepe/miniconda3/envs/ai3i/lib/
2022-12-06 19:50:03.391259: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/volpepe/miniconda3/envs/ai3i/lib/
2022-12-06 19:50:03.391266: W tensorflow/compile

# Setup

Set the following variables before training

In [2]:
USE_MASK    = True
USE_REG     = True

USE_ONE_GPU = True            # or False if another GPU is available
USE_SMALL_GENRE_SET = True    # or False if we want to use the dataset with the full genre subset list
DATASET_NAME = 'tf_data7dict' # or whatever

In [3]:
if USE_MASK and not USE_REG:
    MODEL_NAME = 'mask_only'
elif USE_REG and not USE_MASK:
    MODEL_NAME = 'reg_only'
elif USE_REG and USE_MASK:
    MODEL_NAME = 'reg_and_mask'
else:
    MODEL_NAME = 'baseline'

In [4]:
ROOT_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
conf = config.Config("single_instruments_type", ROOT_PATH, f'model_{MODEL_NAME}')
# Config object has by default the full list of accepted subgenres and works on multi-gpus
# If we use the small dataset
if USE_SMALL_GENRE_SET:
    conf.accepted_subgenres = ['folk', 'nes', 'maestro']

# Training

## Model creation

In [ ]:
if conf.num_devices > 1:
    print("Using multiple GPUs with Mirrored Strategy")
    with conf.training_strategy.scope():
        model = music_model.create_model(num_genres=len(conf.accepted_subgenres), 
                                                    use_masking_layers=USE_MASK,
                                                    use_regularization=USE_REG)
else:
    print("Using single GPU/CPU device")
    model = music_model.create_model(num_genres=len(conf.accepted_subgenres), 
                                                use_masking_layers=USE_MASK,
                                                use_regularization=USE_REG)

In [ ]:
model.summary()

## Loading dataset

In [6]:
dataset = tf.data.Dataset.load(conf.tf_data7dict_path).\
            batch(conf.GLOBAL_BATCH_SIZE).\
            cache().\
            shuffle(conf.SHUFFLE_SIZE).\
            prefetch(conf.PREFETCH_SIZE)

train_dataset = dataset.skip(int(len(dataset)/4))
val_dataset = dataset.skip(int(len(dataset)/8)).take(int(len(dataset)/8))
test_dataset = dataset.take(int(len(dataset)/8))

In [ ]:
X, y = next(dataset.take(1).as_numpy_iterator())
model(X)